# Partie 5. - Moteur de Recherche et visualisation

Dans cette 5ième et dernière partie, nous allons mettre en oeuvre un moteur de recherche pour indexer et rechercher le jeu de données vu en première partie.  Le moteur de recherche que nous utiliserons est OpenSearch (fork Elasticsearch), un moteur de recherche puissant qui est de plus en plus utilisé dans l’industrie ; non seulement pour ses capacités de recherche mais aussi pour ses fonctionnalités de Business Intelligence (BI).

## Import des bibliothèques logicielles et configuration

In [13]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl.query import QueryString, Range, SimpleQueryString
from elasticsearch_dsl import MultiSearch, Search, Q
from datasets import load_dataset # Intégration de jeux de données.

import ast
from tqdm.notebook import tqdm


In [7]:
# Configuration de l'URL vers le moteur de recherche
ES_URL = '127.0.0.1:9200'
ES_USER=""
ES_PWD=""

# Configuration d'un objet qui nous permettra d'indexer des données dans le moteur de recherche.
es = Elasticsearch(ES_URL, http_auth=(ES_USER, ES_PWD))

## 1. Lancement du moteur de recherche avec Docker

Le moteur de recherche devrait déjà être lancé par Docker. Vous pouvez le vérifier en vous rendant sur http://localhost:5601

## 2. Indexation des données

Rechargeons le jeu de données construit dans la Partie 1.

In [15]:
french_book_review_dataset = load_dataset("Abirate/french_book_reviews")

Using custom data configuration Abirate--french_book_reviews-c9c8b5f3107a71dd
Reusing dataset json (/home/vincent/.cache/huggingface/datasets/Abirate___json/Abirate--french_book_reviews-c9c8b5f3107a71dd/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

Ensuite, nous créons une boucle qui pour chaque document l'indexe dans le moteur de recherche avec les paramètres suivants : 
- nom de l'index : `book_review` ;
- identifiant du document : chemin du document (les clés du dictionnaire `documents`) ;
- contenut : JSON du document.

In [39]:
#for doc in documents:
#    es.index(index="book_review",id=doc, body=documents[doc])

for book_title, review, label, rating in tqdm(zip(french_book_review_dataset.data["train"]["book_title"], \
        french_book_review_dataset.data["train"]["reader_review"], \
        french_book_review_dataset.data["train"]["label"], \
        french_book_review_dataset.data["train"]["rating"]), \
        total=len(french_book_review_dataset.data["train"])):
    
    label_str = "Positif"
    if("0" == str(label)):
        label_str = "Négatif"

    json_document = {
        "book_title": str(book_title),
        "review": str(review),
        "label" : str(label_str),
        "rating" : int(rating.as_py())
    }

    es.index(index="book_review",id=str(book_title), body=json_document)


  0%|          | 0/9658 [00:00<?, ?it/s]

Nous avons utilisé les paramètres par défaut pour le pré-traitement du texte, la tokinisation etc. Il est possible de modifier la manière dont les documents sont indexés via les `mappings` : https://opensearch.org/docs/2.0/opensearch/mappings/.

## 3. Exploitation du moteur de recherche

Nous allons maintenant lancer l'interface graphique **OpenDashboards** du moteur de recherche.

1. Lancer l'URL http://localhost:5601/app/management/opensearch-dashboards/objects
2. Cliquer sur "Import" et sélectionner le fichier `opensearch-dashboard.ndjson`
3. Lancer l'interface : http://localhost:5601/app/dashboards
4. Exécuter des requêtes et visualiser les résultats !